In [10]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy
from Viscosity_models import *
from Constants import *
from scipy import stats

In [11]:
dados = pd.read_excel('CaMgSi2O6.xlsx')
dados = dados.sort_values('temp [K]')
x = np.array(dados['temp [K]'])
y_log = np.array(dados['visc [log10 Pa.s]'])

In [12]:
rkf = RepeatedKFold(n_splits=5, n_repeats=6, random_state=2652124)

In [13]:
modelos = pickle.load(open("resultados_ajuste", "rb"))

In [14]:
medias = {}
scores = {}

for equacao, resultado in modelos.items():
    
    scores[equacao.__name__] = []
    medias[equacao.__name__] = 0
    for i, (train_index, test_index) in enumerate(rkf.split(x)):
        
        x_treino = x[train_index]
        y_treino = y_log[train_index]
        y_teste = y_log[test_index]
        x_teste = x[test_index]

        Resultado = Model(equacao).fit(y_treino, resultado.params, T = x_treino)

        y_verdadeiro = y_log[test_index]
        y_previsao = Resultado.eval(T = x_teste)
        RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared = False)
        scores[equacao.__name__].append(RMSE)
    
    medias[equacao.__name__] = np.mean(scores[equacao.__name__])  

    pickle.dump(scores, open("RMSEs", "wb"))

In [18]:
scores

{'VFT': [0.28178202824865184,
  0.3028103597008144,
  0.33224027921694277,
  0.17870352155272834,
  0.33011979776114225,
  0.3610000623852659,
  0.2653491988178729,
  0.276294097599245,
  0.27110966188015445,
  0.2541842576547802,
  0.2185780637566275,
  0.34588687938544005,
  0.2510977046365571,
  0.3043025292592925,
  0.3180476151251757,
  0.26740731108780624,
  0.2937624790984686,
  0.3034664694071004,
  0.2403306323912492,
  0.3440633649158636,
  0.31312246885282985,
  0.27009279254703195,
  0.31331025132071405,
  0.2967585984870791,
  0.25148906373069885,
  0.3442036106215597,
  0.22015829892332717,
  0.2812103156358688,
  0.3708573220761419,
  0.20600714684564272],
 'AM': [0.22271916105231004,
  0.23822216351416758,
  0.2947129566876633,
  0.13104151068143977,
  0.24099552378143532,
  0.32981773476077303,
  0.2253793543885756,
  0.21278692715277686,
  0.19813963780026522,
  0.15590551351271717,
  0.17753993034415985,
  0.28369419896272646,
  0.20168829400936697,
  0.2484571942610

In [15]:
stats.shapiro(score1), stats.shapiro(score2), stats.shapiro(score3), stats.shapiro(score4)

NameError: name 'score1' is not defined

In [ ]:
stats.levene(score1, score2, score3, score4)

In [16]:
testes = {
    'ANOVA' : stats.f_oneway(score1,score2,score3, score4),
    }

print('Significância 0.05')
for teste, estatistica in testes.items():
    pvalor = estatistica.pvalue
    print(f'Para o teste {teste}, o P-valor foi: {pvalor}')

NameError: name 'score1' is not defined

In [9]:
tukey = stats.tukey_hsd(score1,score2,score3, score4)

In [10]:
print(tukey)

Tukey's HSD Pairwise Group Comparisons (95.0% Confidence Interval)
Comparison  Statistic  p-value  Lower CI  Upper CI
 (0 - 1)      0.058     0.000     0.027     0.090
 (0 - 2)      0.043     0.003     0.011     0.074
 (0 - 3)      0.058     0.000     0.027     0.090
 (1 - 0)     -0.058     0.000    -0.090    -0.027
 (1 - 2)     -0.015     0.576    -0.047     0.016
 (1 - 3)      0.000     1.000    -0.031     0.031
 (2 - 0)     -0.043     0.003    -0.074    -0.011
 (2 - 1)      0.015     0.576    -0.016     0.047
 (2 - 3)      0.015     0.576    -0.016     0.047
 (3 - 0)     -0.058     0.000    -0.090    -0.027
 (3 - 1)     -0.000     1.000    -0.031     0.031
 (3 - 2)     -0.015     0.576    -0.047     0.016



In [11]:
resultado = tukey.pvalue

In [12]:
print(resultado)

[[1.00000000e+00 2.38282112e-05 3.01905940e-03 2.38282094e-05]
 [2.38282112e-05 1.00000000e+00 5.75793792e-01 1.00000000e+00]
 [3.01905940e-03 5.75793792e-01 1.00000000e+00 5.75793780e-01]
 [2.38282094e-05 1.00000000e+00 5.75793780e-01 1.00000000e+00]]
